**Задание: улучши нейросеть**

Сборная Москвы по ИИ | Введение в нейронные сети

---

В основном ноутбуке мы обучили простую полносвязную сеть на FashionMNIST и получили ~87% accuracy.

Твоя задача — улучшить модель. Вот несколько идей:

1. **Добавь больше слоёв** — сделай сеть глубже (например, 4-5 линейных слоёв с ReLU)
2. **Поменяй размеры слоёв** — попробуй больше/меньше нейронов (128, 512, ...)
3. **Добавь Dropout** — `nn.Dropout(p=0.2)` между слоями для регуляризации
4. **Поменяй оптимизатор / lr** — попробуй SGD с momentum, или другой learning rate
5. **Обучи дольше** — увеличь число эпох

Цель: попробуй достичь **> 90%** accuracy на тесте.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda


**Загрузка данных** (такая же, как в основном ноутбуке)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

print(f'Train: {len(train_dataset)}, Test: {len(test_dataset)}')

AttributeError: module 'torchvision.datasets' has no attribute 'FashionMNIST'

**Функции для обучения и оценки** (такие же, как в основном ноутбуке)

In [3]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)

    return total_loss / total, correct / total


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        loss = criterion(logits, labels)

        total_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)

    return total_loss / total, correct / total

---

**Базовая модель (для сравнения)**

Это та же модель из основного ноутбука. Запусти её, чтобы получить baseline.

In [4]:
class SimpleNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


baseline = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(baseline.parameters(), lr=1e-3)

print('Обучаем baseline...')
for epoch in range(10):
    train_loss, train_acc = train_epoch(baseline, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(baseline, test_loader, criterion, device)
    print(f'Epoch {epoch+1:2d}/10 | Train acc: {train_acc:.4f} | Test acc: {test_acc:.4f}')

print(f'\nBaseline Test accuracy: {test_acc:.4f}')

Обучаем baseline...
Epoch  1/10 | Train acc: 0.8184 | Test acc: 0.8391
Epoch  2/10 | Train acc: 0.8618 | Test acc: 0.8521
Epoch  3/10 | Train acc: 0.8768 | Test acc: 0.8588
Epoch  4/10 | Train acc: 0.8854 | Test acc: 0.8527
Epoch  5/10 | Train acc: 0.8913 | Test acc: 0.8669
Epoch  6/10 | Train acc: 0.8979 | Test acc: 0.8795
Epoch  7/10 | Train acc: 0.9016 | Test acc: 0.8770
Epoch  8/10 | Train acc: 0.9079 | Test acc: 0.8830
Epoch  9/10 | Train acc: 0.9100 | Test acc: 0.8859
Epoch 10/10 | Train acc: 0.9144 | Test acc: 0.8819

Baseline Test accuracy: 0.8819


---

**Твоя модель**

Напиши свою улучшенную модель ниже. Не забудь:
- Наследоваться от `nn.Module`
- Определить `__init__` и `forward`
- Входной размер: `28 * 28 = 784` (после Flatten)
- Выходной размер: `10` классов (логиты, без softmax)

In [5]:
class ImprovedNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # TODO: определи слои
        pass

    def forward(self, x):
        # TODO: определи forward pass
        pass

In [10]:
# Обучение твоей модели

# my_model = ImprovedNet(num_classes=10).to(device)
# my_criterion = nn.CrossEntropyLoss()
# my_optimizer = torch.optim.Adam(my_model.parameters(), lr=1e-3)

# print(f'Параметров: {sum(p.numel() for p in my_model.parameters()):,}')

# num_epochs = 10
# for epoch in range(num_epochs):
#     train_loss, train_acc = train_epoch(my_model, train_loader, my_criterion, my_optimizer, device)
#     test_loss, test_acc = evaluate(my_model, test_loader, my_criterion, device)
#     print(f'Epoch {epoch+1:2d}/{num_epochs} | '
#           f'Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | '
#           f'Test loss: {test_loss:.4f}, acc: {test_acc:.4f}')

**Сравнение результатов**

После обучения сравни accuracy своей модели с baseline (~87%). Удалось ли побить 89%?